In [23]:
import requests
import pandas as pd
import json
import os 
from dotenv import load_dotenv
import joblib
import sqlite3
import time

In [24]:
load_dotenv()

GECKO_API = os.getenv("GECKO_API")
if not GECKO_API:
    raise ValueError("GECKO_API environment variable is not set.")

In [25]:
network = "eth"
token_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"  # USDT


# Format URL properly
url = f"https://pro-api.coingecko.com/api/v3/onchain/networks/{network}/tokens/{token_address}/pools"

# Get API key from environment variable
API_KEY = os.getenv("GECKO_API")
if not API_KEY:
    raise ValueError("GECKO_API environment variable not set.")

headers = {
    "accept": "application/json",
    "x-cg-pro-api-key": API_KEY
}

# Make request
response = requests.get(url, headers=headers)
response.raise_for_status()
data = response.json()

In [26]:
# All USDT pair pools in the ethereum blockchain

all_pools = []

for pool in data["data"]:
    
    pool_data = {
        "id": pool["id"],
        "name": pool["attributes"]["name"],
        "pool_created_at": pool["attributes"]["pool_created_at"],
        "address": pool["attributes"]["address"]
    }
    all_pools.append(pool_data)

# Convert to DataFrame
df_pools = pd.DataFrame(all_pools)

df_pools.tail()

,id,name,pool_created_at,address
15,eth_0xce93ea3914c62e0008348cf39fd006e130e7c503...,USDe / USDT 0.001%,2025-08-26T11:21:59Z,0xce93ea3914c62e0008348cf39fd006e130e7c503935f...
16,eth_0xd51a44d3fae010294c616388b506acda1bfaae46,USDT / WBTC / WETH,2022-07-13T11:48:12Z,0xd51a44d3fae010294c616388b506acda1bfaae46
17,eth_0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406,DAI / USDT 0.01%,2022-07-14T19:47:16Z,0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406
18,eth_0x9db9e0e53058c89e5b94e29621a205198648425b,WBTC / USDT 0.3%,2021-12-29T13:55:13Z,0x9db9e0e53058c89e5b94e29621a205198648425b
19,eth_0x390f3595bca2df7d23783dfd126427cceb997bf4,crvUSD / USDT,2023-05-18T16:16:53Z,0x390f3595bca2df7d23783dfd126427cceb997bf4


In [27]:
# Converting pool created at to datetime
# For easier calculations, sorting and resampling or grouping by time

# Convert column to datetime (UTC)
df_pools['pool_created_at'] = pd.to_datetime(df_pools['pool_created_at'], utc=True)

# Creating a new column for pool age in days
df_pools['pool_age_days'] = (pd.Timestamp.now(tz='UTC') - df_pools['pool_created_at']).dt.days

df_pools.head()

,id,name,pool_created_at,address,pool_age_days
0,eth_0x72331fcb696b0151904c03584b66dc8365bc63f8...,ETH / USDT 0.05%,2025-01-27 22:08:23+00:00,0x72331fcb696b0151904c03584b66dc8365bc63f8a144...,223
1,eth_0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d4...,USDC / USDT 0.001%,2025-04-01 02:10:47+00:00,0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d44712...,160
2,eth_0x00836fe54625be242bcfa286207795405ca4fd10,sUSDS / USDT,2025-04-18 22:52:10+00:00,0x00836fe54625be242bcfa286207795405ca4fd10,142
3,eth_0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,DAI / USDC / USDT,2022-07-13 11:48:12+00:00,0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,1152
4,eth_0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH / USDT 0.3%,2021-12-29 12:36:12+00:00,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,1348


In [28]:
# Extract fee into its own column (keep the % sign)

df_pools["fee"] = df_pools["name"].str.extract(r"(\d+\.?\d*%)")

# Remove the fee part from the name
df_pools["name"] = df_pools["name"].str.replace(r"\s\d+(\.\d+)?%", "", regex=True)

df_pools.head()

,id,name,pool_created_at,address,pool_age_days,fee
0,eth_0x72331fcb696b0151904c03584b66dc8365bc63f8...,ETH / USDT,2025-01-27 22:08:23+00:00,0x72331fcb696b0151904c03584b66dc8365bc63f8a144...,223,0.05%
1,eth_0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d4...,USDC / USDT,2025-04-01 02:10:47+00:00,0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d44712...,160,0.001%
2,eth_0x00836fe54625be242bcfa286207795405ca4fd10,sUSDS / USDT,2025-04-18 22:52:10+00:00,0x00836fe54625be242bcfa286207795405ca4fd10,142,NaN
3,eth_0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,DAI / USDC / USDT,2022-07-13 11:48:12+00:00,0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,1152,NaN
4,eth_0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH / USDT,2021-12-29 12:36:12+00:00,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,1348,0.3%


In [29]:
#  The joblib directory i want to save extracted pools data to

path = r'/home/realist/projects/DexTracker/backend/Database'
os.makedirs(path, exist_ok=True)

In [30]:
# Save pools dataframe as .pkl
joblib.dump(df_pools, os.path.join(path, "USDT-pairs_pools.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_pools.pkl']

In [31]:
# Not calling for the quote token since its a stablecoin
# base_token_price_usd → For universal USD pricing
# base_token_price_quote_token → For the actual trading rate in the pool

# All USDT pair price in the ethereum blockchain

pair_price = []

for price in data["data"]:
    
    token_data = {
        "name": price["attributes"]["name"],
        "base_price_usd": price["attributes"]["base_token_price_usd"],
        "base_quote_price": price["attributes"]["base_token_price_quote_token"]
    }
    pair_price.append(token_data)

# Convert to DataFrame
df_prices = pd.DataFrame(pair_price)

df_prices.tail()

,name,base_price_usd,base_quote_price
15,USDe / USDT 0.001%,0.995501236841904,1.00464549
16,USDT / WBTC / WETH,0.993591417691551,0.000008924745346
17,DAI / USDT 0.01%,0.997659515725903,0.9946471219
18,WBTC / USDT 0.3%,111598.422612752,111249.321985874
19,crvUSD / USDT,0.992640220850228,1.0053056035


In [32]:
# Save pool price dataframe as .pkl
joblib.dump(df_prices, os.path.join(path, "USDT-pairs_prices.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_prices.pkl']

In [33]:
# TVL (Liquidity for USDT pairs)

pair_tvl = []

for liquidity in data["data"]:
    
    pair_liquidity = {
        "name": liquidity["attributes"]["name"],
        "pair_reserve_in_usd": liquidity["attributes"]["reserve_in_usd"]
    }
    pair_tvl.append(pair_liquidity)

# Convert to DataFrame
df_tvl = pd.DataFrame(pair_tvl)

df_tvl.tail()

,name,pair_reserve_in_usd
15,USDe / USDT 0.001%,2798776.6927
16,USDT / WBTC / WETH,17224871.0417
17,DAI / USDT 0.01%,6902431.1411
18,WBTC / USDT 0.3%,26362676.4064
19,crvUSD / USDT,9829628.4251


In [34]:
# Save pools tvl dataframe as .pkl
joblib.dump(df_tvl, os.path.join(path, "USDT-pairs_tvl.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_tvl.pkl']

In [35]:
# Market metrics (FDV for USDT pairs)

pair_fdv = []

for diluted_value in data["data"]:
    
    pair_fully_diluted_value = {
        "name": diluted_value["attributes"]["name"],
        "fdv_usd": diluted_value["attributes"]["fdv_usd"]
    }
    pair_fdv.append(pair_fully_diluted_value)

# Convert to DataFrame
df_fdv = pd.DataFrame(pair_fdv)

df_fdv.tail()

,name,fdv_usd
15,USDe / USDT 0.001%,86328226409.5858
16,USDT / WBTC / WETH,86228756718.6359
17,DAI / USDT 0.01%,87047762362.6284
18,WBTC / USDT 0.3%,87057256346.5791
19,crvUSD / USDT,86156147271.3765


In [36]:
# Save pools fdv dataframe as .pkl
joblib.dump(df_fdv, os.path.join(path, "USDT-pairs_fdv.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_fdv.pkl']

In [37]:
# Price change

pair_price_change = []

for change in data["data"]:
    attr = change["attributes"]
    price_change = attr.get("price_change_percentage", {})  # This gets the dictionary

    # Create a dictionary combining pool name and price changes
    pool_price_change = {
        "name": attr["name"],
        "m5": float(price_change.get("m5", 0)),
        "m15": float(price_change.get("m15", 0)),
        "m30": float(price_change.get("m30", 0)),
        "h1": float(price_change.get("h1", 0)),
        "h6": float(price_change.get("h6", 0)),
        "h24": float(price_change.get("h24", 0))
    }
    pair_price_change.append(pool_price_change)

# Convert to DataFrame
df_price_chg_pct = pd.DataFrame(pair_price_change)

df_price_chg_pct.tail()

,name,m5,m15,m30,h1,h6,h24
15,USDe / USDT 0.001%,0.0,-0.70,-0.30,-1.04,-0.580,-0.390
16,USDT / WBTC / WETH,0.0,0.00,0.00,0.00,-0.295,-0.303
17,DAI / USDT 0.01%,0.0,0.60,0.00,-0.01,0.010,0.600
18,WBTC / USDT 0.3%,0.0,0.00,0.00,0.00,0.620,0.010
19,crvUSD / USDT,0.0,-0.62,-0.08,-0.17,-0.320,-0.390


In [38]:
# Save pools percentage change in price dataframe as .pkl
joblib.dump(df_price_chg_pct, os.path.join(path, "USDT-pairs_price_chg_pct.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_price_chg_pct.pkl']

In [39]:
pair_transactions = []

for tnxs in data["data"]:
    attr = tnxs["attributes"]
    tx = attr.get("transactions", {})

    pair_tx = {
        "name": attr["name"],
        "buys_5m": tx.get("m5", {}).get("buys", 0),
        "sells_5m": tx.get("m5", {}).get("sells", 0),
        "buyers_5m": tx.get("m5", {}).get("buyers", 0),
        "sellers_5m": tx.get("m5", {}).get("sellers", 0),
        "buys_15m": tx.get("m15", {}).get("buys", 0),
        "sells_15m": tx.get("m15", {}).get("sells", 0),
        "buyers_15m": tx.get("m15", {}).get("buyers", 0),
        "sellers_15m": tx.get("m15", {}).get("sellers", 0),
        "buys_1h": tx.get("h1", {}).get("buys", 0),
        "sells_1h": tx.get("h1", {}).get("sells", 0),
        "buyers_1h": tx.get("h1", {}).get("buyers", 0),
        "sellers_1h": tx.get("h1", {}).get("sellers", 0),
        "buys_24h": tx.get("h24", {}).get("buys", 0),
        "sells_24h": tx.get("h24", {}).get("sells", 0),
        "buyers_24h": tx.get("h24", {}).get("buyers", 0),
        "sellers_24h": tx.get("h24", {}).get("sellers", 0),
    }

    pair_transactions.append(pair_tx)

# Convert to DataFrame
df_transactions = pd.DataFrame(pair_transactions)

df_transactions.tail()

,name,buys_5m,sells_5m,buyers_5m,sellers_5m,buys_15m,sells_15m,buyers_15m,sellers_15m,buys_1h,sells_1h,buyers_1h,sellers_1h,buys_24h,sells_24h,buyers_24h,sellers_24h
15,USDe / USDT 0.001%,0,1,0.0,1.0,0,3,0.0,3.0,3,7,3.0,5.0,76,129,32.0,68.0
16,USDT / WBTC / WETH,0,0,NaN,NaN,1,0,NaN,NaN,6,2,NaN,NaN,355,345,NaN,NaN
17,DAI / USDT 0.01%,0,1,0.0,1.0,1,3,1.0,2.0,2,10,2.0,7.0,100,126,83.0,69.0
18,WBTC / USDT 0.3%,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,11,57,7.0,35.0
19,crvUSD / USDT,1,1,1.0,1.0,1,4,1.0,2.0,8,12,5.0,5.0,90,278,16.0,29.0


In [40]:
# Save pools transaction activity dataframe as .pkl

joblib.dump(df_transactions, os.path.join(path, "USDT-pairs_transactions.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_transactions.pkl']

In [41]:
pair_volume = []

for volumes in data["data"]:
    attr = volumes["attributes"]
    vol = attr.get("volume_usd", {})

    pool_vol = {
        "name": attr["name"],
        "vol_5m": float(vol.get("m5", 0)),
        "vol_15m": float(vol.get("m15", 0)),
        "vol_30m": float(vol.get("m30", 0)),
        "vol_1h": float(vol.get("h1", 0)),
        "vol_6h": float(vol.get("h6", 0)),
        "vol_24h": float(vol.get("h24", 0))
    }

    pair_volume.append(pool_vol)

# Convert to DataFrame
df_volume = pd.DataFrame(pair_volume)

df_volume.tail()

,name,vol_5m,vol_15m,vol_30m,vol_1h,vol_6h,vol_24h
15,USDe / USDT 0.001%,159715.649004,470584.346463,501738.631075,726684.532708,7.382275e+06,2.603505e+07
16,USDT / WBTC / WETH,0.000000,3586.258078,10004.750937,16992.755812,6.256795e+05,2.973233e+06
17,DAI / USDT 0.01%,429956.978029,714001.763206,914761.062422,919674.077484,4.566022e+06,8.181289e+06
18,WBTC / USDT 0.3%,0.000000,0.000000,0.000000,0.000000,9.395485e+05,1.664439e+06
19,crvUSD / USDT,3913.268868,19901.387361,29229.050763,73314.040289,9.842974e+05,2.672145e+06


In [42]:
# Save pools transaction activity dataframe as .pkl

joblib.dump(df_volume, os.path.join(path, "USDT-pairs_volume.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_volume.pkl']

In [43]:
pair_dex = []

for dex_pool in data["data"]:
    dex_info = dex_pool.get("relationships", {}).get("dex", {}).get("data", {})
    pool_dex = {
        "name": pool["attributes"]["name"],
        "dex": dex_info.get("id", "unknown")  # fallback if missing
    }
    pair_dex.append(pool_dex)

df_dex = pd.DataFrame(pair_dex)

df_dex["name"] = df_dex["name"].str.replace(r"\s\d+(\.\d+)?%", "", regex=True)


df_dex.tail()

,name,dex
15,crvUSD / USDT,uniswap-v4-ethereum
16,crvUSD / USDT,curve
17,crvUSD / USDT,uniswap_v3
18,crvUSD / USDT,uniswap_v3
19,crvUSD / USDT,curve


In [44]:
# Save pools transaction activity dataframe as .pkl

joblib.dump(df_dex, os.path.join(path, "USDT-pairs_dex.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_dex.pkl']